In [1]:
import pandas as pd
import mitotrek
import pickle

# Load per-sample mito data and concatenate for each patient

The following sequencing libraries were derived from flow-sorted samples of an NSCLC patient (ID: `SU-L-005`) to maximize cell yield:

| Library ID | Tissue Source | Marker Expression       |
|:-----------|:--------------|:-------------------------|
| 8088       | Lung Tumor     | CD45<sup>+</sup> CD3<sup>+</sup>   |
| 8089       | Lung Tumor     | CD45<sup>+</sup> CD3<sup>-</sup> / CD45<sup>-</sup> |
| 8090       | Lung Normal    | CD45<sup>+</sup> CD3<sup>+</sup>   |
| 8091       | Lung Normal    | CD45<sup>+</sup> CD3<sup>-</sup>   |
| 8092       | Lung Normal    | CD45<sup>-</sup>                |
| 8093       | PBMC           | CD3<sup>+</sup>                 |
| 8094       | PBMC           | CD3<sup>-</sup>                 |

Each library was processed independently using **CellRanger** and **mgatk**, resulting in individual mgatk output files for each.

### Defining Patient-Sample Groupings

We specify the sequencing libraries associated with a given patient using a dictionary, for example:

```python
sample_patient_grouping = {
    "SU-L-005": [
        "8088",  # Lung tumor, CD45+CD3+
        "8089",  # Lung tumor, CD45+CD3-/CD45-
        "8090",  # Lung normal, CD45+CD3+
        "8091",  # Lung normal, CD45+CD3-
        "8092",  # Lung normal, CD45-
        "8093",  # PBMC, CD3+
        "8094",  # PBMC, CD3-
    ]
}
```

> **Note:** Additional patient-sample groupings can be added to this dictionary. The analysis pipeline is designed to iterate through each patient entry and process them independently.


In [2]:
sample_patient_grouping = {'SU-L-005': ['8088', '8089', '8090', '8091', '8092', '8093', '8094']}
sample_patient_map = {pt: k for k, v in sample_patient_grouping.items() for pt in v}

### Mitochondrial Mutation Merging and Filtering

For each patient-sample grouping defined in the `sample_patient_grouping` dictionary, we use the `mitotrek.processing.load_multiple` function to:

1. **Merge all mgatk outputs** associated with the same patient.
2. **Perform mitochondrial mutation quality control (QC) and filtering** using the **union method** under the hood.

> **Union Filtering Strategy:**  
> A mitochondrial variant is retained **in all associated samples** if it passes QC criteria in **any one** of the samples. This inclusive strategy ensures that potentially informative variants are not missed due to sample-specific filtering artifacts.


In [3]:
mito_file_path_temp = './mgatk_output/'
pt_hetero_dict, pt_variant_summary_dict = dict(), dict()

for pt in sample_patient_grouping:
    cur_pt_dir = mito_file_path_temp+pt+'/'
    pt_variant_summary_dict[pt], pt_hetero_dict[pt] = mitotrek.processing.load_multiple(cur_pt_dir, sample_patient_grouping[pt])
    print()

2594 variants detected.
8088: 7515 cells, 509 variants.
8089: 4410 cells, 422 variants.
8090: 5662 cells, 347 variants.
8091: 5045 cells, 377 variants.
8092: 7053 cells, 1220 variants.
8093: 16713 cells, 1021 variants.
8094: 7737 cells, 867 variants.



In [4]:
# save the patient-merged mgatk output
pickle.dump(pt_hetero_dict, open('output/pt_hetero_dict.p', 'wb'))
pickle.dump(pt_variant_summary_dict, open('output/pt_variant_summary_dict.p', 'wb'))

# Clone calling

In [5]:
# OPTIONAL: load list of cell barcodes that pass scATAC QC and discard cells not in this whitelist
with open('scatac_whitelist.txt', 'r') as f:
    scatac_whitelist = f.readlines()
    scatac_whitelist = pd.Index([x.strip() for x in scatac_whitelist])

After QC filtering, mitochondrial variants are further processed to infer **clonal populations** using co-occurrence patterns across single cells.

The main parameters controlling this conversion are:

- **`max_abundance`**  
  *Type:* `float`  
  *Description:* The maximum allowable fraction of cells in which a variant can be detected.  
  *Purpose:* Filters out non-informative, ubiquitous variants that may represent germline polymorphisms or sequencing artifacts. Variants detected in more than `max_abundance` of cells are excluded from clonal analysis.

- **`bin_cut_off`**  
  *Type:* `float`  
  *Description:* Heteroplasmy threshold used to binarize the heteroplasmy matrix per variant per cell.  
  *Purpose:* Converts continuous heteroplasmy values into binary calls (`present`/`absent`) to minimize noise.

- **`corr_cutoff`**  
  *Type:* `float`  
  *Description:* Minimum correlation coefficient required to merge two variants into the same clone.  
  *Purpose:* Ensures that only strongly co-occurring variants are grouped together, improving clone specificity.

We recommend using default parameters unless there is a special use case.

In [6]:
# convert variants to clones
per_pt_cell_per_clone = dict()
per_pt_clone_marker = dict()
for pt in pt_hetero_dict:
    print('{}...'.format(pt))
    cur_hetero_df = pt_hetero_dict[pt]
    cur_hetero_df = cur_hetero_df.loc[cur_hetero_df.index.intersection(scatac_whitelist)]
    per_pt_cell_per_clone[pt], per_pt_clone_marker[pt] = mitotrek.core.assign_cell_to_clones(cur_hetero_df, max_abundance=0.05, bin_cutoff=0.09, corr_cutoff=0.7)[:2]
    
    # prepend clone name with pt id
    per_pt_cell_per_clone[pt] = pt + ':' + per_pt_cell_per_clone[pt]
    for old_clone_key in list(per_pt_clone_marker[pt].keys()):
        new_clone_key = pt + ':' + old_clone_key
        per_pt_clone_marker[pt][new_clone_key] = per_pt_clone_marker[pt].pop(old_clone_key)

    print()

SU-L-005...
42041 cells in heteroplasmy df.
23372 cells available to be assigned.
2345 variants used to call clones...
9779 cells assigned to more than one clone and discarded.
8 clones defined by more than one variant.
13562 cells assigned to 2156 clones.



### Output Data Structures

After variant-to-clone conversion, the following two output dictionaries capture the clonal assignments and their defining mutations:

#### `per_pt_cell_per_clone`

- **Type:** `dict[str, pd.Series]`
- **Description:** Maps cells to inferred clone identities for each patient.
- **Structure:**
  ```python
  {
      "patient_id_1": pd.Series({
          "cell_1": "clone_1",
          "cell_2": "clone_2",
          ...
      }),
      ...
  }
  ```
- **Notes:**
  - Each key corresponds to a patient ID.
  - The associated `pd.Series` maps **cell barcodes** to their assigned **`clone_id`**.
  - Only includes cells that have been assigned to a clone.

#### `per_pt_clone_marker`

- **Type:** `dict[str, dict[str, list[str]]]`
- **Description:** Stores the defining mitochondrial variants for each clone per patient.
- **Structure:**
  ```python
  {
      "patient_id_1": {
          "clone_1": ["variant_A", "variant_B"],
          "clone_2": ["variant_C"],
          ...
      },
      ...
  }
  ```
- **Notes:**
  - Each outer dictionary key is a patient ID.
  - The inner dictionary maps each `clone_id` to a list of **mitochondrial variants** that define that clone.
  - These variants are the result of filtering and correlation-based merging.

These output files are then integrated with scATAC-seq data for downstream analysis using any single-cell data analysis software of choice (eg. ArchR).

In [7]:
# save output files
pickle.dump(per_pt_cell_per_clone, open('output/per_pt_cell_per_clone.p', 'wb'))
pickle.dump(per_pt_clone_marker, open('output/per_pt_clone_marker.p', 'wb'))